In [1]:
import numpy as np
import math
import statsmodels.api as sm
import imblearn
from tqdm.notebook import tqdm
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite, community 
from numpy import nansum
from numpy import nanmean
from numpy.random import choice
import itertools
from networkx.algorithms import community
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
cb1 = pd.read_csv('cb1.csv')
cb2 = pd.read_csv('cb2.csv')
cb3 = pd.read_csv('cb3.csv')


In [3]:
cb = cb1.append(cb2)
cb = cb.append(cb3)
cb.index = range(len(cb))
cb['date_my'] = (pd.to_datetime(cb['date'],format='%m/%d/%Y', errors='coerce')).dt.to_period('m')
cb['date_y'] = (pd.to_datetime(cb['date'],format='%m/%d/%Y', errors='coerce')).dt.to_period('y')

nodes_org = cb[['target_name', 'target_ID', 'target_country', 'target_continent','industry', 'target_founded', 'target_status']].drop_duplicates()
nodes_org['bipartite'] = 1
nodes_inv = cb[['investors_name', 'investor_id', 'investor_country', 'investor_continent', 'investor_type', 'investor_founded']].drop_duplicates()
nodes_org = nodes_org.rename(columns={"target_name": "node", "target_country": "country",  
                                      "target_continent": "continent", "industry":"industry", "target_status":"status",
                                     "target_ID":"index", "target_founded":"foundation"})
nodes_inv = nodes_inv.rename(columns={"investors_name": "node","investor_country": "country", 
                                      "investor_continent": "continent", "investor_type":"industry","investor_id":'index', "investor_founded":"foundation" })
nodes_inv['bipartite'] = 0
nodes = nodes_org.append(nodes_inv, ignore_index= True)
nodes['id'] = nodes.index
nodes_org = nodes[nodes['bipartite']==1]
nodes_inv = nodes[nodes['bipartite']==0]
numbers = cb.groupby(['target_name','stage'], as_index = False).count()[['target_name','stage','investors_name']].rename(columns={"investors_name": "investor_numbers"})
cb_new = pd.merge(cb, numbers,  how='left', left_on=['target_name','stage'], right_on = ['target_name','stage'])
cb_new = pd.merge(cb_new, nodes[nodes['bipartite']== 1],  how='left', left_on=['target_name','target_ID'], right_on = ['node','index'])
cb_new = pd.merge(cb_new, nodes[nodes['bipartite']== 0],  how='left', left_on=['investors_name','investor_id'], right_on = ['node','index'])


cb_new.date_y=cb_new.date_y.dt.year


In [4]:
B = nx.MultiGraph(name='org-bid')
# Add nodes with the node attribute "bipartite"
B.add_nodes_from(nodes[nodes.bipartite==0].id)
B.add_nodes_from(nodes[nodes.bipartite==1].id)
nx.set_node_attributes(B, pd.Series(nodes.node, index=nodes.id).to_dict(), 'name')
nx.set_node_attributes(B, pd.Series(nodes.country, index=nodes.id).to_dict(), 'country')
nx.set_node_attributes(B, pd.Series(nodes.continent, index=nodes.continent).to_dict(), 'continent')
nx.set_node_attributes(B, pd.Series(nodes.industry, index=nodes.id).to_dict(), 'industry')
nx.set_node_attributes(B, pd.Series(nodes.foundation, index=nodes.id).to_dict(), 'foundation')
nx.set_node_attributes(B, pd.Series(nodes.bipartite, index=nodes.id).to_dict(), 'bipartite')
nx.set_node_attributes(B, pd.Series(nodes.status, index=nodes.id).to_dict(), 'target_status')
# Add edges only between nodes of opposite node sets


In [5]:
cb_new_wo = cb_new[(cb_new["round_simp"] != 'Line of Credit') & (cb_new["round_simp"]!= 'Loan') & (cb_new["round_simp"]!= 'Debt')]
cb_new_wo = cb_new_wo[pd.notna(cb_new_wo['date_y'])]
cb_new_wo = cb_new_wo[pd.notna(cb_new_wo['size_real'])]

cb_new_wo.index = range(len(cb_new_wo))
for i in range(len(cb_new_wo)):
    B.add_edge(cb_new_wo.loc[i,'id_y'], cb_new_wo.loc[i,'id_x'], date=(cb_new_wo.loc[i,'date_y']), m_raised=cb_new_wo.loc[i,'size_real'], 
               investors = cb_new_wo.loc[i,'investor_numbers'], round_simp =cb_new_wo.loc[i,'round_simp'])

In [6]:
brutti = [x for x,y in B.nodes(data=True) if (y['name'] in ['Undisclosed Investors', 'ciao']) or (y['name']=='Dead')or (y['name']=='Public')
          or  (y['name']=='Undisclosed Angel Investors') or (y['name']=='Undisclosed Venture Investors')]
B.remove_nodes_from(brutti)


In [388]:
brutti2 = [(u,v) for u,v,d in B.edges(data=True) if (d['round_simp'] =="Debt")]

In [389]:
brutti2

[]

In [7]:
stati = cb_new.groupby('id_x')
stati2 = stati.tail(1).reset_index(drop=True)
stati3 = stati2[stati2["date"].notna()]

In [238]:
nodes[nodes['id'] == 7903]

,node,index,country,continent,industry,foundation,status,bipartite,id
7903,CoImmune,25848.0,United States,North America,Pharmaceuticals / Drugs,1997.0,Alive / Active,1,7903


In [35]:
stati2[stati2["target_status"] == "Merged"].drop_duplicates()[["target_name","round_simp","target_status", "date"]]

,target_name,round_simp,target_status,date
3692,Lankenau Institute for Medical Research,Merger,Merged,01/01/1985
4176,Global Pharmaceutical,IPO,Merged,12/19/1995
4240,BioClin International,Merger,Merged,08/19/1996
4256,Gensia Pharmaceuticals,Merger,Merged,11/01/1996
4320,OccuSystems,Merger,Merged,08/29/1997
...,...,...,...,...
49604,Hamilton County Systems of Care,Merger,Merged,07/15/2021
49605,Partnership For A Healthy Hamilton County,Merger,Merged,07/15/2021
49645,Fillmaster,Merger,Merged,07/16/2021
49646,FLAVORx,Merger,Merged,07/16/2021


In [8]:
fun_status = stati2[["round_simp", "target_status"]].groupby(['round_simp', "target_status"]).size().unstack(fill_value=0)[1:61]

In [45]:
fun_status.T.index.values

array(['Acquired', 'Alive / Active', 'Assets Purchased',
       'Dead / Inactive', 'IPO / Went public', 'Merged'], dtype=object)

In [9]:
fun_status.mode()['Dead / Inactive']
stage = list(fun_status.idxmax(axis=1).index)
stage.append("Acq - P2P")

stato = list(fun_status.idxmax(axis=1))
stato.append("Acquired")

In [93]:
set(stato)

{'Acquired',
 'Alive / Active',
 'Assets Purchased',
 'Dead / Inactive',
 'IPO / Went public',
 'Merged'}

In [10]:
def get_status(rounds):
    a = list()
    for i in rounds:
        indice = stage.index(i)
        a.append(stato[indice])
    return a
    
    


In [71]:
get_status(["Acquisition"])

['Acquired']

In [67]:
stage.index("Acquisition")

0

In [13]:
cb_new["stato"] = get_status(list(cb_new["round_simp"]))


In [96]:
cb_new["stato"]

0         IPO / Went public
1         IPO / Went public
2         IPO / Went public
3         IPO / Went public
4         IPO / Went public
                ...        
187799             Acquired
187800       Alive / Active
187801    IPO / Went public
187802             Acquired
187803             Acquired
Name: stato, Length: 187804, dtype: object

In [ ]:
stati.tail(1)

In [ ]:

remove=['Line of Credit',  'Loan']

cb_new=cb_new.loc[~cb_new.round_simp.isin(remove)]

In [ ]:
cb_new[14720:14724]

In [ ]:
#--------------------------------------

In [14]:
id_round = cb_new[['target_ID','round_simp']]
id_round = id_round.drop_duplicates()
id_round.index = range(len(id_round))
id_round['id_round'] = id_round.index
id_round

id_year = cb_new[['target_ID','date_y']]
id_year = id_year.drop_duplicates()
id_year.index = range(len(id_year))
id_year['id_year'] = id_year.index
id_year

id_stage = cb_new[['target_ID','stage']]
id_stage = id_stage.drop_duplicates()
id_stage.index = range(len(id_stage))
id_stage['id_stage'] = id_stage.index
id_stage

cb_new = pd.merge(cb_new, id_round,  how='left', left_on=['target_ID','round_simp'], right_on = ['target_ID','round_simp'])
cb_new = pd.merge(cb_new, id_stage,  how='left', left_on=['target_ID','stage'], right_on = ['target_ID','stage'])
cb_new = pd.merge(cb_new, id_year,  how='left', left_on=['target_ID','date_y'], right_on = ['target_ID','date_y'])

In [15]:
##bidbid in stesso round
links = []
subcb = cb_new[['round_simp','date_y','id_y','size_real','id_round','investor_numbers','target_name','id_x','stato','target_country']].drop_duplicates()
subcb = subcb[pd.notna(subcb['date_y'])]
subcb = subcb[pd.notna(subcb['size_real'])]
subcb = subcb[(subcb["round_simp"] != 'Line of Credit') & (subcb["round_simp"]!= 'Loan') & (subcb["round_simp"]!= 'Debt')]
for i in tqdm(subcb['id_round'].unique()):
    subcb2 = subcb[subcb['id_round']==i]
    for couple in itertools.combinations_with_replacement(np.unique(subcb2['id_y'].values),2):
        links.append((couple + ({'date': subcb2['date_y'].values[0],
                                'round': subcb2['round_simp'].values[0],
                                'money': subcb2['size_real'].values[0]/(subcb2['investor_numbers'].values[0]),
                                'investor_numbers': subcb2['investor_numbers'].values[0],
                                'status': subcb2["stato"].values[0],
                                 'id_target': subcb2["id_x"].values[0],
                                 'country_target': subcb2["target_country"].values[0],
                                'target_name': subcb2['target_name'].values[0]},)))

In [140]:
links

[(50685,
  50685,
  {'date': 1976,
   'round': 'Seed VC',
   'money': 491039.4265232975,
   'investor_numbers': 1,
   'status': 'Alive / Active',
   'id_target': 6693,
   'country_target': 'United States',
   'target_name': 'Genentech'}),
 (50989,
  50989,
  {'date': 1981,
   'round': 'Seed VC',
   'money': 2994535.5191256832,
   'investor_numbers': 1,
   'status': 'Alive / Active',
   'id_target': 6708,
   'country_target': 'United States',
   'target_name': 'Vestar Research'}),
 (54093,
  54093,
  {'date': 1981,
   'round': 'Seed VC',
   'money': 1026766.5952890792,
   'investor_numbers': 2,
   'status': 'Alive / Active',
   'id_target': 6710,
   'country_target': 'United States',
   'target_name': 'Diagnostic Technology'}),
 (54093,
  54094,
  {'date': 1981,
   'round': 'Seed VC',
   'money': 1026766.5952890792,
   'investor_numbers': 2,
   'status': 'Alive / Active',
   'id_target': 6710,
   'country_target': 'United States',
   'target_name': 'Diagnostic Technology'}),
 (54094,
  

In [254]:
bidbid_stage_self=nx.MultiGraph()
bidbid_stage_self.add_nodes_from(nodes[nodes.bipartite==0].id)
nx.set_node_attributes(bidbid_stage_self, pd.Series(nodes_inv.node, index=nodes_inv.id).to_dict(), 'name')
nx.set_node_attributes(bidbid_stage_self, pd.Series(nodes_inv.country, index=nodes_inv.id).to_dict(), 'country')
nx.set_node_attributes(bidbid_stage_self, pd.Series(nodes_inv.continent, index=nodes_inv.id).to_dict(), 'continent')
nx.set_node_attributes(bidbid_stage_self, pd.Series(nodes_inv.industry, index=nodes_inv.id).to_dict(), 'industry')
nx.set_node_attributes(bidbid_stage_self, pd.Series(nodes_inv.foundation, index=nodes_inv.id).to_dict(), 'foundation')
nx.set_node_attributes(bidbid_stage_self, pd.Series(nodes_inv.bipartite, index=nodes_inv.id).to_dict(), 'bipartite')
bidbid_stage_self.add_edges_from(links) 
remove = [node for node,degree in dict(bidbid_stage_self.degree()).items() if degree == 0]
bidbid_stage_self.remove_nodes_from(remove)
bidbid_stage_self.remove_nodes_from(brutti)
remove2 = [node for node,degree in dict(bidbid_stage_self.degree()).items() if degree == 0]
bidbid_stage_self.remove_nodes_from(remove2)
nx.readwrite.write_gpickle(bidbid_stage_self,'bidbid_stage_self_cb.gpickle')
nx.info(bidbid_stage_self)

'Name: \nType: MultiGraph\nNumber of nodes: 20206\nNumber of edges: 335734\nAverage degree:  33.2311'

In [ ]:
bidbid_stage_self

In [267]:
bidbid_stage_self_single = nx.Graph()
for u,v,data in tqdm(bidbid_stage_self.edges(data=True)):
    d = data['date']
    m = data['money']
    r = data['round']
    t = data['target_name']
    
    if bidbid_stage_self_single.has_edge(u,v):
        bidbid_stage_self_single[u][v]['date'] = list(set([bidbid_stage_self[u][v][0]['date']] + [d]))
        bidbid_stage_self_single[u][v]['money'] += m
        bidbid_stage_self_single[u][v]['rounds'] = list(set([bidbid_stage_self[u][v][0]['round']] + [r]))
        bidbid_stage_self_single[u][v]['target_name'] = list(set([bidbid_stage_self[u][v][0]['target_name']] + [t]))
    else:
        bidbid_stage_self_single.add_edge(u, v, date=[d])
        bidbid_stage_self_single.add_edge(u, v, money = m)
        bidbid_stage_self_single.add_edge(u, v, rounds =[r])
        bidbid_stage_self_single.add_edge(u, v, target_name = [t])
        

In [262]:
sorted(bidbid_stage_self.degree, key=lambda x: x[1], reverse=True)

[(51253, 4600),
 (50667, 3523),
 (50481, 2858),
 (50770, 2719),
 (51084, 2511),
 (50595, 2454),
 (50917, 2284),
 (51077, 2252),
 (51153, 2245),
 (55245, 2147),
 (57908, 2109),
 (55129, 2031),
 (50919, 1966),
 (56017, 1956),
 (53091, 1909),
 (51276, 1877),
 (55127, 1829),
 (56185, 1823),
 (51305, 1777),
 (50605, 1734),
 (50939, 1730),
 (50516, 1702),
 (50766, 1667),
 (50859, 1616),
 (51204, 1609),
 (50685, 1560),
 (54057, 1526),
 (55133, 1486),
 (52127, 1483),
 (53343, 1471),
 (51263, 1448),
 (56477, 1415),
 (50519, 1401),
 (50515, 1397),
 (50577, 1396),
 (54696, 1387),
 (51005, 1386),
 (56015, 1381),
 (54697, 1312),
 (50683, 1293),
 (61604, 1256),
 (50668, 1214),
 (51304, 1209),
 (50768, 1187),
 (56152, 1177),
 (50928, 1130),
 (55131, 1088),
 (50480, 1080),
 (52419, 1075),
 (56933, 1075),
 (50531, 1073),
 (62781, 1070),
 (51175, 1066),
 (50710, 1062),
 (50985, 1044),
 (51211, 1041),
 (51512, 1036),
 (50585, 1034),
 (51063, 1028),
 (57366, 1021),
 (52117, 1017),
 (59069, 1016),
 (50893,

In [266]:
nodes[nodes['id']==57908]

,node,index,country,continent,industry,foundation,status,bipartite,id
57908,RA Capital Management,13415.0,United States,North America,Advisory,2002.0,NaN,0,57908


In [152]:
nx.readwrite.write_gpickle(bidbid_stage_self_single,'bidbid_stage_self_single.gpickle')
nx.info(bidbid_stage_self_single)

'Name: \nType: Graph\nNumber of nodes: 20413\nNumber of edges: 164243\nAverage degree:  16.0920'

In [268]:
anni = range(1995,2022)
for periodo in anni:
    locals()["bidbid_stage_self_single_"+str(periodo-1)]=nx.Graph([(u,v,d) for u,v,d in bidbid_stage_self_single.edges(data=True) if d['date'][-1] < periodo])
    print(periodo-1)

1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [269]:
for periodo in anni:
    nx.readwrite.write_gpickle(locals()["bidbid_stage_self_single_"+str(periodo-1)], "bidbid_stage_self_single_"+str(periodo-1)+'.gpickle')
    print(nx.info(locals()["bidbid_stage_self_single_"+str(periodo-1)]))

Name: 
Type: Graph
Number of nodes: 386
Number of edges: 2614
Average degree:  13.5440
Name: 
Type: Graph
Number of nodes: 489
Number of edges: 3244
Average degree:  13.2679
Name: 
Type: Graph
Number of nodes: 570
Number of edges: 3802
Average degree:  13.3404
Name: 
Type: Graph
Number of nodes: 674
Number of edges: 4160
Average degree:  12.3442
Name: 
Type: Graph
Number of nodes: 1186
Number of edges: 11397
Average degree:  19.2192
Name: 
Type: Graph
Number of nodes: 1641
Number of edges: 18369
Average degree:  22.3876
Name: 
Type: Graph
Number of nodes: 2135
Number of edges: 24483
Average degree:  22.9349
Name: 
Type: Graph
Number of nodes: 2559
Number of edges: 30571
Average degree:  23.8929
Name: 
Type: Graph
Number of nodes: 2916
Number of edges: 36021
Average degree:  24.7058
Name: 
Type: Graph
Number of nodes: 3282
Number of edges: 44337
Average degree:  27.0183
Name: 
Type: Graph
Number of nodes: 3600
Number of edges: 51427
Average degree:  28.5706
Name: 
Type: Graph
Number of 

In [167]:
for u,v,d in bidbid_stage_self_single.edges(data=True):
    a = (d['date'][-1])

In [169]:
a<2021

False

In [194]:
for periodo in anni:
    print((locals()["bidbid_stage_self_single_"+str(periodo-1)]).number_of_edges())

2601
2775
2939
3113
4299
5797
12817
17345
20923
26594
35291
39897
44816
49089
52958
56634
60996
64978
68503
71842
75826
81543
89950
99178
112173
124587
144157


In [186]:
sorted(bidbid_stage_self_single_2020.degree, key=lambda x: x[1], reverse=True)

[(51253, 964),
 (50667, 742),
 (50481, 681),
 (55245, 572),
 (50919, 571),
 (50917, 569),
 (50770, 563),
 (50595, 544),
 (51276, 518),
 (50516, 499),
 (51077, 479),
 (53091, 471),
 (51084, 458),
 (51153, 452),
 (51305, 445),
 (56185, 432),
 (55127, 430),
 (51204, 429),
 (56017, 429),
 (55129, 423),
 (50766, 421),
 (50939, 420),
 (50531, 417),
 (54696, 415),
 (54697, 414),
 (50605, 412),
 (50685, 409),
 (50515, 407),
 (50859, 407),
 (50577, 405),
 (50519, 399),
 (51005, 397),
 (55133, 382),
 (51263, 368),
 (50874, 366),
 (52127, 365),
 (54057, 361),
 (51709, 357),
 (50683, 352),
 (50710, 351),
 (57908, 351),
 (51304, 348),
 (56477, 347),
 (56015, 345),
 (51211, 343),
 (50602, 337),
 (50832, 336),
 (51063, 329),
 (50554, 326),
 (51021, 323),
 (50505, 321),
 (50668, 321),
 (56967, 321),
 (50985, 317),
 (51175, 311),
 (54828, 310),
 (55131, 308),
 (54787, 298),
 (51512, 296),
 (50893, 296),
 (52419, 292),
 (50803, 291),
 (50581, 289),
 (52002, 286),
 (54704, 284),
 (57366, 283),
 (50768, 2

In [193]:
nodes[nodes['id'] == 50515]

,node,index,country,continent,industry,foundation,status,bipartite,id
50515,Abingworth,22678.0,United Kingdom,Europe,Venture Capital,NaN,NaN,0,50515


In [ ]:
#------------------------------------------------

In [18]:
bid_nodes = {n for n, d in B.nodes(data=True) if d["bipartite"] == 0}
org_nodes = set(B) - bid_nodes

# ORGORG2

In [16]:
from itertools import islice

def window(seq, n):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [19]:
count=0
finestra = list(range(1990,2022))
win = []
for i in range(1,len(finestra)+1):
    win.append(finestra[:i])
for year in range(1979,2022):
    locals()["orgorg4_"+str(year)] = nx.MultiGraph()

for node in tqdm(bid_nodes):
    ego=nx.ego_graph(B, node)
    ndx=list(ego.nodes())
    ndx.remove(node)
    for periodo in win: 
        subndx=[x for x in ndx if [y for y in list(ego.edges(data=True)) if y[0]==x or y[1]==x][0][2]['date'] in periodo]
        for couple in itertools.combinations(subndx, 2):
            anno1 = ego[node][couple[0]][0]['date']
            anno2 = ego[node][couple[1]][0]['date']
            if anno1 - anno2 in range(-6,7):
                locals()["orgorg4_"+str(periodo[-1])].add_edge(couple[0], couple[1], anno = max(anno1,anno2))
    count+=1
    if count%1000==0:
        print(count, end=' ')

1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 


In [20]:
#tasformo i periodini multi in periodini single
for periodo in win:
    a = nx.Graph()
    for u,v,data in locals()["orgorg4_"+str(periodo[-1])].edges(data=True):
        d = data['anno']
        if a.has_edge(u,v)==False:
            a.add_edge(u, v, date=[d])        
    locals()["orgorg4_single_"+str(periodo[-1])] = a
    nx.readwrite.write_gpickle(locals()["orgorg4_single_"+str(periodo[-1])], "orgorg4_single"+str(periodo[-1])+".gpickle")
    print(nx.info(locals()["orgorg4_single_"+str(periodo[-1])]))
    print(periodo[-1])

Name: 
Type: Graph
Number of nodes: 38
Number of edges: 43
Average degree:   2.2632
1990
Name: 
Type: Graph
Number of nodes: 82
Number of edges: 140
Average degree:   3.4146
1991
Name: 
Type: Graph
Number of nodes: 136
Number of edges: 353
Average degree:   5.1912
1992
Name: 
Type: Graph
Number of nodes: 179
Number of edges: 653
Average degree:   7.2961
1993
Name: 
Type: Graph
Number of nodes: 205
Number of edges: 879
Average degree:   8.5756
1994
Name: 
Type: Graph
Number of nodes: 234
Number of edges: 1060
Average degree:   9.0598
1995
Name: 
Type: Graph
Number of nodes: 260
Number of edges: 1192
Average degree:   9.1692
1996
Name: 
Type: Graph
Number of nodes: 309
Number of edges: 1517
Average degree:   9.8188
1997
Name: 
Type: Graph
Number of nodes: 563
Number of edges: 3577
Average degree:  12.7069
1998
Name: 
Type: Graph
Number of nodes: 871
Number of edges: 8212
Average degree:  18.8565
1999
Name: 
Type: Graph
Number of nodes: 1325
Number of edges: 18221
Average degree:  27.5034

In [427]:
periodo

[1990]

In [425]:
win

[[1990],
 [1990, 1991],
 [1990, 1991, 1992],
 [1990, 1991, 1992, 1993],
 [1990, 1991, 1992, 1993, 1994],
 [1990, 1991, 1992, 1993, 1994, 1995],
 [1990, 1991, 1992, 1993, 1994, 1995, 1996],
 [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997],
 [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998],
 [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999],
 [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000],
 [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001],
 [1990,
  1991,
  1992,
  1993,
  1994,
  1995,
  1996,
  1997,
  1998,
  1999,
  2000,
  2001,
  2002],
 [1990,
  1991,
  1992,
  1993,
  1994,
  1995,
  1996,
  1997,
  1998,
  1999,
  2000,
  2001,
  2002,
  2003],
 [1990,
  1991,
  1992,
  1993,
  1994,
  1995,
  1996,
  1997,
  1998,
  1999,
  2000,
  2001,
  2002,
  2003,
  2004],
 [1990,
  1991,
  1992,
  1993,
  1994,
  1995,
  1996,
  1997,
  1998,
  1999,
  2000,
  2001,
  2002,
  2003,
  2004,
  2005],
 [1990,
  1991,
  1992,
  19

In [423]:
nx.info(orgorg4_single_2020)

'Name: \nType: MultiGraph\nNumber of nodes: 16871\nNumber of edges: 719245\nAverage degree:  85.2641'

In [421]:
list(range(-6,7))

[-6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6]

In [418]:
periodo[-1]

2021

In [414]:
nx.info(orgorg2_2021)

'Name: \nType: MultiGraph\nNumber of nodes: 10846\nNumber of edges: 379726\nAverage degree:  70.0214'

In [420]:
#tasformo i periodini multi in periodini single
finestra = window(list(range(1979,2022)), 7)
for periodo in finestra:
    a = nx.Graph()
    for u,v,data in locals()["orgorg2_"+str(periodo[-1])].edges(data=True):
        d = data['anno']
        if a.has_edge(u,v)==False:
            a.add_edge(u, v, date=[d])        
    locals()["orgorg2_single_"+str(periodo[-1])] = a
    nx.readwrite.write_gpickle(locals()["orgorg2_single_"+str(periodo[-1])], "orgorg2_single"+str(periodo[-1])+".gpickle")
    print(periodo[-1])

1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


Exception ignored in: <function tqdm.__del__ at 0x000001A096BA4AF0><object repr() failed>
Traceback (most recent call last):
  File "C:\Users\Marco\anaconda3\lib\site-packages\tqdm\std.py", line 1086, in __del__
    self.close()
KeyboardInterrupt: 


2021


# ORGORG3

In [392]:
count=0
finestra = list(range(1990,2022))
win = []
for i in range(1,len(finestra)+1):
    win.append(finestra[:i])
for year in range(1979,2022):
    locals()["orgorg3_"+str(year)] = nx.MultiGraph()

for node in tqdm(bid_nodes):
    ego=nx.ego_graph(B, node)
    ndx=list(ego.nodes())
    ndx.remove(node)
    for periodo in win: 
        subndx=[x for x in ndx if [y for y in list(ego.edges(data=True)) if y[0]==x or y[1]==x][0][2]['date'] in periodo]
        for couple in itertools.combinations(subndx, 2):
            anno1 = ego[node][couple[0]][0]['date']
            anno2 = ego[node][couple[1]][0]['date']
            locals()["orgorg3_"+str(periodo[-1])].add_edge(couple[0], couple[1], anno = max(anno1,anno2))
    count+=1
    if count%1000==0:
        print(count, end=' ')

1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 


In [394]:
nx.info(orgorg3_2020)

'Name: \nType: MultiGraph\nNumber of nodes: 17036\nNumber of edges: 1033542\nAverage degree: 121.3362'

In [400]:
list(bid_nodes)[1650:1654]

[52115, 52116, 52117, 52118]

In [407]:
cb_new[cb_new['id_y'] == 52117]

,target_name,stage,size_MLN,date,investors_name,url,description,industry_x,target_country,target_collections,...,continent_y,industry,foundation_y,status_y,bipartite_y,id_y,stato,id_round,id_stage,id_year
3689,Contraline,Seed,0.02,NaN,Y Combinator,contraline.com,Contraline is a biotechnology company developi...,Biotechnology,United States,Biopharmaceuticals,...,North America,Incubator/Accelerator,2005.0,NaN,0,52117,Alive / Active,2488,2494,2020
3987,VitroLabs,Other Investors,NaN,NaN,Y Combinator,vitrolabsinc.com,VitroLabs is a 3D tissue engineering company.,Biotechnology,United States,Regenerative Medicine,...,North America,Incubator/Accelerator,2005.0,NaN,0,52117,Alive / Active,2710,2717,2636
4486,Apprentice Health,Other Investors,NaN,NaN,Y Combinator,apprenticehealth.com,Apprentice Health is a predictive analytics fi...,Internet Software & Services,United States,"Digital Health, Value-Based Care & Population ...",...,North America,Incubator/Accelerator,2005.0,NaN,0,52117,Alive / Active,3090,3097,3009
6531,1910 Genetics,Seed VC,4.10,NaN,Y Combinator,1910genetics.com,1910 Genetics provides computational medicines...,Drug Development,United States,Artificial Intelligence,...,North America,Incubator/Accelerator,2005.0,NaN,0,52117,Alive / Active,4900,4911,4767
7747,AOA Dx,Incubator/Accelerator,NaN,NaN,Y Combinator,aoadx.com,"AOA Dx bring innovative, impactful and cost-ef...",Medical Product Distribution,United States,Medical Devices,...,North America,Incubator/Accelerator,2005.0,NaN,0,52117,Alive / Active,6021,6038,5839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185467,SFA Therapeutics,Seed,0.13,07/12/2021,Y Combinator,sfatherapeutics.com,SFA Therapeutics is working on a new advanceme...,Drug Development,United States,Microbiome,...,North America,Incubator/Accelerator,2005.0,NaN,0,52117,Alive / Active,91384,107351,93972
187268,Encellin,Seed VC,5.90,08/05/2021,Y Combinator,encellin.com,Encellin is developing a thin-film cell encaps...,Biotechnology,United States,Biopharmaceuticals,...,North America,Incubator/Accelerator,2005.0,NaN,0,52117,Alive / Active,92019,108118,94628
187318,Infiuss Health,Seed,0.13,08/06/2021,Y Combinator,infiuss.com,Infiuss Health democratizes remote access to p...,Internet Software & Services,Cameroon,Digital Health,...,North America,Incubator/Accelerator,2005.0,NaN,0,52117,Alive / Active,92041,108148,94653
187319,Yemaachi Biotechnology,Seed,0.13,08/06/2021,Y Combinator,yemaachi.com,Yemaachi Biotechnology is a biotech company th...,Disease Diagnosis,Ghana,"Medical Devices, Cancer and Health & Wellness ...",...,North America,Incubator/Accelerator,2005.0,NaN,0,52117,Alive / Active,92042,108149,94654


In [373]:
ego=nx.ego_graph(B, 50515)

In [374]:
ndx = list(ego.nodes())

In [381]:
ego[50515][21510][0]

{'date': 2015,
 'm_raised': 74235253.27869546,
 'investors': 5,
 'round_simp': 'Series B'}

In [377]:
subndx = [x for x in ndx if [y for y in list(ego.edges(data=True)) if y[0]==x or y[1]==x][0][2]['date'] in range(1970,2000)]

In [222]:
x = 19489

In [227]:
[y for y in list(ego.edges(data=True)) if y[0]==x or y[1]==x]

[(19489,
  50515,
  {'date': 2013,
   'm_raised': 53171074.59043611,
   'investors': 7,
   'round_simp': 'Series A'}),
 (19489,
  50515,
  {'date': 2015,
   'm_raised': 46096710.56228734,
   'investors': 11,
   'round_simp': 'Series B'}),
 (19489,
  50515,
  {'date': 2016,
   'm_raised': 18471702.56513972,
   'investors': 12,
   'round_simp': 'Series B'}),
 (19489,
  50515,
  {'date': 2017,
   'm_raised': 43296217.45537908,
   'investors': 12,
   'round_simp': 'Series C'})]

In [212]:
[y for y in list(ego.edges(data=True))][0][2]

{'date': 2015,
 'm_raised': 74235253.27869546,
 'investors': 5,
 'round_simp': 'Series B'}